Import Panda und SQLalchemy

In [ ]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
import psycopg2
import numpy as np

from update import update_fkz
from update import update_group
from update import update_sql_table

csv-File importieren.

In [ ]:
df_bln = pd.read_csv("data/Berlin_22-23.csv", delimiter= ",", encoding= 'utf-8')

In [ ]:
df_bln.head()

Columns names anpassen, Sonderzeichen entfernen

In [ ]:
df_bln.columns = df_bln.columns.str.replace(" ", "_")
df_bln.columns = df_bln.columns.str.lower()

Drop der Columns, die nicht gebraucht werden

In [ ]:
df_bln = df_bln.drop(['id','typ', 'bezeichnung', 'bereich', 'bereichsbezeichnung', 
                    'hauptgruppe','hauptgruppenbezeichnung', 'obergruppe', 'obergruppenbezeichnung',
                    'oberfunktion', 'oberfunktionsbezeichnung', 'hauptfunktion', 
                    'hauptfunktionsbezeichnung'
], axis=1)

Values ins passende Format setzten und leading-0 einfügen, wo sie automatisch entfernt wurde

In [ ]:
df_bln['einzelplan'] = df_bln['einzelplan'].astype(str)
df_bln['kapitel'] = df_bln['kapitel'].astype(str)
df_bln['funktion'] = df_bln['funktion'].astype(str)
df_bln['titel'] = df_bln['titel'].astype(str)

df_bln['titel'] = df_bln['titel'].apply(lambda x:x.zfill(5))
df_bln['einzelplan'] = df_bln['einzelplan'].apply(lambda x:x.zfill(2))
df_bln['kapitel'] = df_bln['kapitel'].str[-2:]
df_bln['kapitel'] = df_bln['kapitel'].apply(lambda x:x.zfill(3))
df_bln['fkz'] = df_bln['funktion'].apply(lambda x:x.zfill(3))
df_bln.head()

Titel in gruppe und counter gesplittet, gruppe und counter eingefügt

In [ ]:
df_bln["gruppe"] = df_bln["titel"].str[:3]
df_bln["counter"] = df_bln["titel"].str[3:]
df_bln

jahr vorhanden -  state_id eingefügt

In [ ]:

df_bln["state_id"] =  3



anhand der titelart einnahmen und ausgaben in den Zahlen definiert (ginge auch mit gruppe)
titel gedropt, da redundant

In [ ]:

df_bln.loc[df_bln['titelart'] == 'Einnahmetitel', 'amount'] = df_bln['betrag']
df_bln.loc[df_bln['titelart'] == 'Ausgabetitel', 'amount'] = -df_bln['betrag']

df_bln = df_bln.drop(['titel','titelart', 'betragtyp'], axis=1)
df_bln = df_bln.drop(['funktion'], axis=1)


einmal angucken, da schön

In [ ]:
df_bln.head(5)

Umbenennen von spalten für sql

In [ ]:
df_bln.rename(columns = {'einzelplan' : 'ep', 'titelbezeichnung' : 'zweckbestimmung', 
                        'jahr' : 'year', 'kapitelbezeichnung' : 'kapitel_txt', 
                        'einzelplanbezeichnung' : 'ep_txt', 'gruppenbezeichnung' : 'gruppe_txt',
                        'funktionsbezeichnung' : 'fkz_txt'
                        }, inplace = True)

df_budget für den Import in budget_all aufbereiten

In [ ]:
df_budget = df_bln[['ep', 'kapitel', 'fkz', 'zweckbestimmung', 'amount', 
'gruppe', 'counter', 'year', 'state_id']]



df_ep für den Import in einzelplaene aufbereiten

In [ ]:

df_ep = df_bln.drop_duplicates(['ep', 'year'])
df_ep = df_ep[['ep', 'ep_txt', 'year', 'state_id']]
df_ep

df_kapitel für den Import in kapitel aufbereiten

In [ ]:
df_kapitel = df_bln.drop_duplicates(['ep', 'kapitel', 'year'])
df_kapitel = df_kapitel[['ep', 'kapitel', 'kapitel_txt', 'year', 'state_id']]

In [ ]:
df_kapitel

Wir bereiten alles für den Push in die Database vor.

In [ ]:
get_sql_config()
engine = get_engine()

Puschen

In [ ]:
#push_to_database(df_budget, "budget_all", engine , "capstone_public_budgeting")

In [ ]:
table_name = 'einzelplaene'
update_sql_table(df_ep,table_name)
#push_to_database(df_ep, "einzelplaene", engine , "capstone_public_budgeting")

In [ ]:
table_name = 'kapitel'
update_sql_table(df_kapitel,table_name)
#push_to_database(df_kapitel, "kapitel", engine , "capstone_public_budgeting")

Gruppe und Funktionsbezeichnungen checken

In [ ]:
df_gruppe = df_bln.drop_duplicates(['gruppe', 'gruppe_txt'])
df_gruppe = df_gruppe[['gruppe', 'gruppe_txt']]
#update_group(df_gruppe)

In [ ]:
df_funktion = df_bln.drop_duplicates(['fkz', 'fkz_txt'])
df_funktion = df_funktion[['fkz', 'fkz_txt']]
update_fkz(df_funktion)

In [ ]:
df_gruppe